In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from summa import keywords
from tqdm import tqdm

In [2]:
CV=5
cv_df = pd.DataFrame(index=range(CV * 5))

In [71]:
nona_train_df = train_df.dropna()

In [3]:
train_df = pd.read_csv("./nona_train.csv")
#test_df = pd.read_csv("./dataset/test.csv",escapechar = "\\", quoting = 3)
#submission_df = pd.read_csv("./dataset/sample_submission.csv",escapechar = "\\", quoting = 3)

In [9]:
train_df.columns

Index(['Unnamed: 0', 'TITLE', 'DESCRIPTION', 'BULLET_POINTS', 'BRAND',
       'BROWSE_NODE_ID'],
      dtype='object')

#### Data Pre Processsing: removal of less useful words
#### Feature selection: unigram/bigram models of the featureset using top occuring terms of each category levels, Reduce Cardinaliity
#### Feature encoding one-hot encoding, targer(mean) encoding
#### Multi-class classification models: Distance based, SVM, Decission trees

In [11]:
train_df = train_df.drop(columns='Unnamed: 0',axis=1)

In [12]:
#check nan values
train_df.isna().sum()

TITLE             0
DESCRIPTION       0
BULLET_POINTS     0
BRAND             0
BROWSE_NODE_ID    0
dtype: int64

In [15]:
train_df.head(10)

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,"[Color: Blue,Sleeve: Full Sleeve,Material: Cot...",Bhavya Enterprise,5
3,Glance Women's Wallet (Black) (LW-21),This Black wallet by Glance will be a treasure...,[The Most Comfortable Women's Wallet That You ...,Glance,6
4,Wild Animals Hungry Brain Educational Flash Ca...,Wild Animals are the animals that mostly stays...,[Playful learning: Flash cards develops the lo...,hungry brain,7
5,Hindi Barakhadi Educational Flash Cards for Ki...,Hindi vowel signs and sounds are very importan...,[Playful learning: Flash cards develops the lo...,hungry brain,7
6,White : Braun KF400-WH Aromaster 10-Cup Coffee...,Color:White Product description 10 Cup Carafe ...,"[Water-dispersal system provides fast, uniform...",Braun,11
7,"Jiffy Steamer with Plastic Steam Head (Beige, ...","Jiffy J-2000 clothes steamer, 1300 watts, with...",[1300 watt solid brass heating element (120 vo...,Jiffy,14
8,OXO Good Grips BPA-Free Turkey Baster with Cle...,<b>OXO Good Grips Turkey Baster</b><br><br><ul...,[Unique bulb shape and collar for two comforta...,OXO,16
9,Westcott 5684 5 x 6 Feet Collapsible Backdrop ...,Westcott offers many styles and colors of qual...,"[Effortlessly opens and collapses in seconds,B...",Westcott,18


In [13]:
train_df['BROWSE_NODE_ID'].value_counts()

1045       208992
5           44483
1052        37721
1295        31128
374         29891
            ...  
1860637         1
2062081         1
161185          1
2036668         1
1705928         1
Name: BROWSE_NODE_ID, Length: 9636, dtype: int64

## Punctuation removal

In [21]:
from rake_nltk import Rake

In [ ]:
def remove_punctuation(text):
    punctuationfree="".join([str(i) for i in text if str(i) not in string.punctuation])
    return punctuationfree
def lower(text):
    return text.lower()
def tokenize(text):
    return re.split('W+',text)

stopwords = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

porter_stemmer = PorterStemmer()
def stemming(text):
    stem_text = [porter_stemmer.stem(word) for word in text]
    return stem_text

wordnet_lemmatizer = WordNetLemmatizer()
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text

<ipython-input-74-5e6110cc703a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nona_train_df['TITLE'] = nona_train_df['TITLE'].apply(lambda x:remove_punctuation(x))


In [22]:
rake_key = Rake()
def keywords(text):
    rake_key.extract_keywords_from_text(text)
    keyword_extracted = rake_key.get_ranked_phrases()
    return keyword_extracted

In [28]:
for i in range(len(train_df)):
    train_df.iloc[i]['TITLE'] = kef_bonus = pd.ywords(train_df.iloc[i]['TITLE'])
    train_df.iloc[i]['DESCRIPTION'] = keywords(train_df.iloc[i]['DESCRIPTION'])
    train_df.iloc[i]['BULLET_POINTS'] = keywords(train_df.iloc[i]['BULLET_POINTS'])

<ipython-input-28-6d59300a3ac7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.iloc[i]['TITLE'] = keywords(train_df.iloc[i]['TITLE'])
<ipython-input-28-6d59300a3ac7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.iloc[i]['DESCRIPTION'] = keywords(train_df.iloc[i]['DESCRIPTION'])
<ipython-input-28-6d59300a3ac7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.iloc[i]['BULLET_POINTS'] = keywords(train_df.il

In [30]:
train_df.iloc[0]['TITLE'] = keywords(train_df.iloc[0]['TITLE'])

<ipython-input-30-32f96fe01be1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.iloc[0]['TITLE'] = keywords(train_df.iloc[0]['TITLE'])


In [33]:
train_df.loc[0]['TITLE'] = keywords(train_df.iloc[0]['TITLE'])

<ipython-input-33-f7650ead0728>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.loc[0]['TITLE'] = keywords(train_df.iloc[0]['TITLE'])


In [34]:
train_df.iloc[0]['TITLE']

'Pete The Cat Bedtime Blues Doll, 14.5 Inch'

In [57]:
one_label = train_df[train_df['BROWSE_NODE_ID']==1]
for i in one_label.index:
    print(keywords.keywords(one_label['TITLE'][i], words=5).split('\n'))

['magnet', 'yorker', 'refrigerator', 'nyhm', 'x']
['kitchen', 'prints', 'clean', 'style', 'dirty']
['typography', 'cm', 'fridge', 'circular', 'magnet']
['pc', 'latch', 'wardrobe', 'touch', 'cupboard']
['fridge', 'orange', 'temperature', 'magnet', 'sg']
['purple', 'block', 'printed', 'room', 'living']
['combo', 'quality', 'pack', 'vprint', 'magnet']
['locker', 'comforts', 'refrigerator', 'mall', 'magnet']
['wildlife', 'artisans', 'society', 'coorg', 'embossed']
['cafe', 'birthday', 'yaya', 'gifts', 'mummy']
['souvenir', 'india', 'magnet', 'fridge', 'taj']
['home', 'magnet', 'cafe', 'fridge', 'printed']
['light', 'x', 'green', 'badges', 'texture']
['magnet', 'neodymium', 'recovery', 'sea']
['magnets', 'square', 'mdf']
['wood', 'multipurpose', 'rays', 'seven', 'happiness']
['eco', 'corner', 'material']
['lock', 'hio', 'open', 'frame']
['fridge', 'kaala', 'rajinikanth', 'karikala', 'magnet']
['blackboard', 'wall', 'cum', 'x inches']
['whiteboard', 'erase', 'dry', 'magnetic', 'sunshine']
['

IndexError: list index out of range

In [39]:
train_df['TITLE'][80]

'Barbie Fairytopia Magic of the Rainbow Mix & Switch Fairy: Dragonfly'